In [1]:
import cv2
import os
import json
import matplotlib.pylab as plt

In [2]:
def read_first_frame(fp):
    cap = cv2.VideoCapture(fp)
    ret, cropped = cap.read()
    if cropped is None:
        cap = cv2.VideoCapture(".".join(fp.split(".")[:-1]) + ".mp4")
        ret, cropped = cap.read()
        print("changed to mp4")
    return cropped, cap

In [ ]:
path = "../../data/soft_tissue_study"
class_folders = [
    "Abscess/AbscessVideos", "Cellulitis/CellulitisVideos", "Normal/NormalVideos",
]
crop_output_file = "../../data/crop.json"
# To access properties of a VideoCapture in openCV, we use an enum referenced  here
# https://docs.opencv.org/3.4/d4/d15/group__videoio__flags__base.html#gaeb8dd9c89c10a5c63c139bf7c4f5704d
CAP_PROP_FRAME_COUNT = 7


with open(crop_output_file, "r") as infile:
    crop_dir = json.load(infile)

try:
    for folder in class_folders:
        for filename in os.listdir(os.path.join(path, folder)):
            print(filename)
            if os.path.join(folder, filename) in crop_dir.keys():
                print("already done")
                continue
            if filename[0] == ".":
                continue

            print(os.path.exists(os.path.join(path, folder, filename)))
            cap = cv2.VideoCapture(os.path.join(path, folder, filename))
            ret, frame = cap.read()
            print(ret)

            x, y, _ = frame.shape
            min_shape = max([x, y])
            crop = [0, 0, min_shape]
            while True:
                bottom, left, size = crop
                plt.imshow(frame[bottom:bottom + size, left:left + size])
                plt.show()
                crop_in = input("okay?")
                if crop_in == 1 or crop_in == "1":
                    crop_dir[os.path.join(folder,
                                        filename)] = [crop, [0, cap.get(CAP_PROP_FRAME_COUNT)]]
                    break
                if crop_in == "0":
                    ret, frame = cap.read()
                else:
                    crop_in = input("input list " + str(crop))
                    print(crop_in)
                    crop = eval(crop_in)
                    print(crop)

except SyntaxError:
    print("invalid input, stopping here but will attempt to preserve the already entered crops")
                

with open(crop_output_file, "w") as infile:
    json.dump(crop_dir, infile)